In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('/content/gdrive/MyDrive/BDA_project/코딩/df_ver1.csv')
df2 = pd.read_csv('/content/gdrive/MyDrive/BDA_project/코딩/df_ver2.csv')

## 최종 train 파일 만들기
- s/m 와 m/s 로 전처리한 각각의 파일을 합치기
- 가중치를 고려한 최종 next_arrive_time를 업데이트 하기

In [2]:
df1.drop(columns = 'Unnamed: 0', inplace =True)
df2.drop(columns = 'Unnamed: 0', inplace =True)

In [3]:
df1

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,m/s,time
0,0,2019-10-15,405136001,7997025,360-1,제주대학교입구,06시,266.000000,제대마을,24,Tuesday,0.090226,11.083333,morning
1,1,2019-10-15,405136001,7997025,360-1,제대마을,06시,333.265436,제대아파트,36,Tuesday,0.108108,9.250000,morning
2,2,2019-10-15,405136001,7997025,360-1,제대아파트,06시,420.903308,제주대학교,40,Tuesday,0.096386,10.375000,morning
3,3,2019-10-15,405136001,7997025,360-1,남국원(아라방면),06시,578.000000,제주여자중고등학교(아라방면),42,Tuesday,0.072664,13.761905,morning
4,4,2019-10-15,405136001,7997025,360-1,도호동,07시,322.581998,은남동,64,Tuesday,0.171123,5.843750,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205956,210452,2019-10-28,405328102,7983486,281-2,비석거리,21시,526.655462,삼아아파트,96,Monday,0.181818,5.500000,evening
205957,210453,2019-10-28,405328102,7983486,281-2,동문로터리,21시,280.000000,매일올레시장 7번입구,50,Monday,0.178571,5.600000,evening
205958,210454,2019-10-28,405328102,7983486,281-2,서귀포시 구 버스터미널,21시,114.000000,아랑조을거리 입구,16,Monday,0.140351,7.125000,evening
205959,210455,2019-10-28,405328102,7983486,281-2,아랑조을거리 입구,21시,219.256538,평생학습관,38,Monday,0.170404,5.868421,evening


In [4]:
df2

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,m/s,time
0,0,2019-10-15,405136001,7997025,360-1,제주대학교입구,06시,266.000000,제대마을,24,Tuesday,0.090226,11.083333,morning
1,1,2019-10-15,405136001,7997025,360-1,제대마을,06시,333.265436,제대아파트,36,Tuesday,0.108108,9.250000,morning
2,2,2019-10-15,405136001,7997025,360-1,제대아파트,06시,420.903308,제주대학교,40,Tuesday,0.096386,10.375000,morning
3,3,2019-10-15,405136001,7997025,360-1,남국원(아라방면),06시,578.000000,제주여자중고등학교(아라방면),42,Tuesday,0.072664,13.761905,morning
4,4,2019-10-15,405136001,7997025,360-1,도호동,07시,322.581998,은남동,64,Tuesday,0.171123,5.843750,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208619,210452,2019-10-28,405328102,7983486,281-2,비석거리,21시,526.655462,삼아아파트,96,Monday,0.181818,5.500000,evening
208620,210453,2019-10-28,405328102,7983486,281-2,동문로터리,21시,280.000000,매일올레시장 7번입구,50,Monday,0.178571,5.600000,evening
208621,210454,2019-10-28,405328102,7983486,281-2,서귀포시 구 버스터미널,21시,114.000000,아랑조을거리 입구,16,Monday,0.140351,7.125000,evening
208622,210455,2019-10-28,405328102,7983486,281-2,아랑조을거리 입구,21시,219.256538,평생학습관,38,Monday,0.170404,5.868421,evening


In [23]:
df_final = df1.merge(df2, on = ['id','date', 'route_id', 'vh_id', 'route_nm', 'now_station',
               'now_arrive_time', 'next_station', 'next_arrive_time','week', 'time'], how = 'inner')

In [24]:
df_final.drop(columns = ['s/m_y', 'distance_y', 'm/s_x'], inplace = True)

In [26]:
df_final.rename(columns = {'s/m_x' : 's/m', 'm/s_y' : 'm/s', 'distance_x' :'distance'}, inplace = True)

In [27]:
df_final

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,time,m/s
0,0,2019-10-15,405136001,7997025,360-1,제주대학교입구,06시,266.000000,제대마을,24,Tuesday,0.090226,morning,11.083333
1,1,2019-10-15,405136001,7997025,360-1,제대마을,06시,333.265436,제대아파트,36,Tuesday,0.108108,morning,9.250000
2,2,2019-10-15,405136001,7997025,360-1,제대아파트,06시,420.903308,제주대학교,40,Tuesday,0.096386,morning,10.375000
3,3,2019-10-15,405136001,7997025,360-1,남국원(아라방면),06시,578.000000,제주여자중고등학교(아라방면),42,Tuesday,0.072664,morning,13.761905
4,4,2019-10-15,405136001,7997025,360-1,도호동,07시,322.581998,은남동,64,Tuesday,0.171123,morning,5.843750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204123,210452,2019-10-28,405328102,7983486,281-2,비석거리,21시,526.655462,삼아아파트,96,Monday,0.181818,evening,5.500000
204124,210453,2019-10-28,405328102,7983486,281-2,동문로터리,21시,280.000000,매일올레시장 7번입구,50,Monday,0.178571,evening,5.600000
204125,210454,2019-10-28,405328102,7983486,281-2,서귀포시 구 버스터미널,21시,114.000000,아랑조을거리 입구,16,Monday,0.140351,evening,7.125000
204126,210455,2019-10-28,405328102,7983486,281-2,아랑조을거리 입구,21시,219.256538,평생학습관,38,Monday,0.170404,evening,5.868421


# next arrive time 업데이트

In [35]:
df = pd.read_csv('/content/gdrive/MyDrive/BDA_project/코딩/df_ver0.csv')

In [36]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [37]:
q3 = df['s/m'].quantile(0.75)
q1 = df['s/m'].quantile(0.25)
q = (q3-q1)*1.5 +q3

In [38]:
q

0.4406365884977709

In [39]:
df_out = df[df['s/m'] >= q]
df_out = df_out.reset_index(drop=True)

확인용

In [40]:
df_out[df_out['id'] ==34225]

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,m/s,time
1972,34225,2019-10-17,405136012,7997033,360-12,탐라도서관,20시,518.0,제주고등학교/중흥S클래스,2168,Thursday,4.185328,0.23893,evening


In [41]:
outlier_index = df_out['id']
outlier_index = outlier_index.to_list()

In [42]:
for idx in outlier_index:
    # 각 행에 대한 조건을 검사하여 불리언 시리즈 생성
    condition = df_final['id'] == idx

    # 조건을 만족하는 행에 대해서 'next_arrive_time' 값을 계산
    df_final.loc[condition, 'next_arrive_time'] = df_final.loc[condition, 's/m'] * df_final.loc[condition, 'distance']

In [44]:
df_final[df_final['id'] ==34225]

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,time,m/s
33197,34225,2019-10-17,405136012,7997033,360-12,탐라도서관,20시,518.0,제주고등학교/중흥S클래스,227.414634,Thursday,0.439024,evening,0.23893


In [46]:
df_final.to_csv('df_final.csv')